In [1]:
!pip install pycm livelossplot albumentations
%pylab inline

     |████████████████████████████████| 57 kB 1.5 MB/s eta 0:00:011
     |████████████████████████████████| 547 kB 7.1 MB/s eta 0:00:01
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
Populating the interactive namespace from numpy and matplotlib


In [2]:
use_albumentations = False

train_whole_model = True
train_last_layer = False
train_some_layers = False

use_googlenet = False
use_resnet50 = True
use_resnet18 = False

In [3]:
from sklearn.metrics import accuracy_score # this allows us to evaluate our model at every iteration
from sklearn.metrics import f1_score # this allows us to evaluate our validation accuracy
from sklearn.model_selection import StratifiedShuffleSplit # this allows us to create a random validation split

# These imports help plot the convergence and create the confusion matrix
from livelossplot import PlotLosses
from pycm import *

# These imports help us create models and datasets
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, random_split
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

# This allows me to create my own custom dataset
from torch.utils.data import Dataset 
from torchvision.datasets.folder import *

# This allows me to import pretrained models for transfer learning
import torchvision.models as models

# This allows me to do a number of transforms for data augmentation later on
from torchvision.transforms import Compose, ToTensor, ColorJitter, Resize, Normalize, RandomApply, RandomChoice, RandomRotation, RandomCrop, RandomResizedCrop, RandomHorizontalFlip, RandomAffine, ToPILImage

# These imports help us write the submission file
import json, csv
import os
import os.path

# We will be using albumentations to perform data augmentation
if (use_albumentations):
    from albumentations import Compose
    import albumentations.augmentations.transforms as transforms

# This helps us keep a copy of model state dicts
import copy

# To display random images
from random import randrange

# Enable hardware acceleration
device = 'cpu'
if torch.cuda.device_count() > 0 and torch.cuda.is_available():
    print("Cuda installed! Running on GPU!")
    device = 'cuda'
else:
    print("No GPU available!")

Cuda installed! Running on GPU!


In [4]:
# The means of our entire training set, calculated in another file:
means = [0.4805, 0.4483, 0.3978]
stds = [0.2177, 0.2138, 0.2136]

In [14]:
transform_resnet_test = Compose([
    Resize(299),
    ToTensor(),
    Normalize(means, stds)
]) 

transform_resnet_train = Compose([
    #ToPILImage(),
    RandomAffine(0, translate=(0.1, 0.1)),
    RandomHorizontalFlip(),
    RandomCrop(size=(64, 64), padding = 6),
    Resize(224),
    ToTensor(),
    Normalize(mean=means, std=stds), 
]) 


In [6]:
class ImageFolderWithPaths(ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """

    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

data_dir = "../input/acse-miniproject/test/"
test_set = ImageFolderWithPaths(data_dir, transform = transform_resnet_test) # our custom dataset
test_loader = DataLoader(test_set)

In [7]:
seed = 42
lr = 1e-2
momentum = 0.5
batch_size = 64
test_batch_size = 100
n_epochs = 30

In [15]:
my_data = ImageFolder("../input/acse-miniproject/train/", transform = transform_resnet_test)

train_size = int(0.95 * len(my_data))
validation_size = len(my_data) - train_size
train_dataset, validation_dataset = random_split(my_data, [train_size, validation_size])
#train_dataset = my_data

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
validation_loader = DataLoader(validation_dataset, batch_size=test_batch_size, shuffle=False, num_workers=0)

In [19]:
# Trains our model on the training set
def train(model, optimizer, criterion, data_loader):
    model.train()
    train_loss, train_accuracy = 0, 0
    for X, y in data_loader:
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        a2, aux = model(X.view(-1, 3, 299, 299))
        loss1 = criterion(a2, y)
        loss2 = criterion(aux, y)
        loss = loss1 + 0.4 * loss2
        loss.backward()
        train_loss += loss*X.size(0)
        y_pred = F.log_softmax(a2, dim=1).max(1)[1]
        train_accuracy += accuracy_score(y.cpu().numpy(), y_pred.detach().cpu().numpy())*X.size(0)
        optimizer.step()  
        
    return train_loss/len(data_loader.dataset), train_accuracy/len(data_loader.dataset)
  
# Checks our model against the validation set
def validate(model, criterion, data_loader):
    model.eval()
    validation_loss, validation_accuracy = 0., 0.
    for X, y in data_loader:
        with torch.no_grad():
            X, y = X.to(device), y.to(device)
            a2= model(X.view(-1, 3, 299, 299))
            loss = criterion(a2, y)
            validation_loss += loss*X.size(0)
            y_pred = F.log_softmax(a2, dim=1).max(1)[1]
            validation_accuracy += accuracy_score(y.cpu().numpy(), y_pred.cpu().numpy())*X.size(0)
            
    return validation_loss/len(data_loader.dataset), validation_accuracy/len(data_loader.dataset)
  
# Evaluates our model's % accuracy
def evaluate(model, data_loader):
    model.eval()
    ys, y_preds = [], []
    for X, y in data_loader:
        with torch.no_grad():
            X, y = X.to(device), y.to(device)
            a2 = model(X.view(-1, 3, 299, 299))
            y_pred = F.log_softmax(a2, dim=1).max(1)[1]
            ys.append(y.cpu().numpy())
            y_preds.append(y_pred.cpu().numpy())
            
    return np.concatenate(y_preds, 0),  np.concatenate(ys, 0)

# Generates predictions in the required format
def predict(model, data_loader):
    model.eval()
    files, y_preds = [], []
    for X, y, z in data_loader:
        with torch.no_grad():
            X, y = X.to(device), y.to(device)
            a2 = model(X.view(-1, 3, 299, 299))
            y_pred = F.log_softmax(a2, dim=1).max(1)[1]
            y_preds.append(y_pred.cpu().numpy())
            files.append(z)
            
    return np.concatenate(y_preds, 0), np.concatenate(files, 0)

# Generates predictions by averaging two models
def multimodel_predict(model_first, model_second, data_loader):
    model_first.eval()
    model_second.eval()
    files, y_preds = [], []
    for X, y, z in data_loader:
        with torch.no_grad():
            X, y = X.to(device), y.to(device)
            a2 = model_first(X.view(-1, 3, 224, 224))
            a3 = model_second(X.view(-1, 3, 224, 224))
            a2 = (a2 + a3) / 2.
            y_pred = F.log_softmax(a2, dim=1).max(1)[1]
            y_preds.append(y_pred.cpu().numpy())
            files.append(z)
            
    return np.concatenate(y_preds, 0), np.concatenate(files, 0)

# Generates predictions by averaging 3 models
def threemodel_predict(model_first, model_second, model_third, data_loader):
    model_first.eval()
    model_second.eval()
    model_third.eval()
    files, y_preds = [], []
    for X, y, z in data_loader:
        with torch.no_grad():
            X, y = X.to(device), y.to(device)
            a2 = model_first(X.view(-1, 3, 224, 224))
            a3 = model_second(X.view(-1, 3, 224, 224))
            a4 = model_third(X.view(-1, 3, 224, 224))
            a2 = (a2 + a3 + a4) / 3.
            y_pred = F.log_softmax(a2, dim=1).max(1)[1]
            y_preds.append(y_pred.cpu().numpy())
            files.append(z)
            
    return np.concatenate(y_preds, 0), np.concatenate(files, 0)

# Checks the accuracy of our 3 model evaluation
def threemodel_evaluate(model_first, model_second, model_third, data_loader):
    model_first.eval()
    model_second.eval()
    model_third.eval()
    files, y_preds = [], []
    for X, y in data_loader:
        with torch.no_grad():
            X, y = X.to(device), y.to(device)
            a2 = model_first(X.view(-1, 3, 224, 224))
            a3 = model_second(X.view(-1, 3, 224, 224))
            a4 = model_third(X.view(-1, 3, 224, 224))
            a2 = (a2 + a3 + a4) / 3.
            y_pred = F.log_softmax(a2, dim=1).max(1)[1]
            y_preds.append(y_pred.cpu().numpy())
            
    return np.concatenate(y_preds, 0)

In [10]:
'''if (use_googlenet):
    model = models.googlenet(pretrained=True).to(device)
elif (use_resnet18):
    model = models.resnet18(pretrained=True).to(device)
else:
    model = models.densenet121(pretrained=True).to(device)

num_ftrs = model.classifier.in_features
model.classifier = nn.Linear(num_ftrs, 200) # change number of output classes

#num_ftrs = model.fc.in_features
#model.fc = nn.Linear(num_ftrs, 200) # change number of output classes

# uncomment to load previously created model:
# model.load_state_dict(torch.load("./Googlenet_full_barely_augmented.pth"))'''

'if (use_googlenet):\n    model = models.googlenet(pretrained=True).to(device)\nelif (use_resnet18):\n    model = models.resnet18(pretrained=True).to(device)\nelse:\n    model = models.densenet121(pretrained=True).to(device)\n\nnum_ftrs = model.classifier.in_features\nmodel.classifier = nn.Linear(num_ftrs, 200) # change number of output classes\n\n#num_ftrs = model.fc.in_features\n#model.fc = nn.Linear(num_ftrs, 200) # change number of output classes\n\n# uncomment to load previously created model:\n# model.load_state_dict(torch.load("./Googlenet_full_barely_augmented.pth"))'

In [11]:
model = models.inception_v3(pretrained=True, transform_input=True).to(device)

#num_ftrs = model.classifier.in_features
#model.classifier = nn.Linear(num_ftrs, 200) # change number of output classes

num_ftrs = model.AuxLogits.fc.in_features
model.AuxLogits.fc = nn.Linear(num_ftrs, 200)

print(num_ftrs)

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 200) # change number of output classes

Downloading: "https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth" to /root/.cache/torch/checkpoints/inception_v3_google-1a9a5a14.pth



768


In [12]:
if (train_some_layers):
    ct = 0
elif (train_last_layer):
    ct = -100
else:
    ct = 12
    
for child in model.children():
    ct += 1
    if ct < 12:
        for param in child.parameters():
            param.requires_grad = False
    else:
        for param in child.parameters():
            param.requires_grad = True
            
print(ct)

if (train_last_layer):
    for param in model.fc.parameters():
        param.requires_grad = True

30


In [16]:
model = model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=1e-3)
criterion = nn.CrossEntropyLoss()

liveloss = PlotLosses()
for epoch in range(40):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    logs = {}
    train_loss, train_accuracy = train(model, optimizer, criterion, train_loader)

    logs['' + 'log loss'] = train_loss.item()
    logs['' + 'accuracy'] = train_accuracy.item()
    
    validation_loss, validation_accuracy = validate(model, criterion, validation_loader)
    logs['val_' + 'log loss'] = validation_loss.item()
    logs['val_' + 'accuracy'] = validation_accuracy.item()
    
    if validation_accuracy.item() > best_acc:
        best_acc = validation_accuracy.item()
        best_model_wts = copy.deepcopy(model.state_dict())
    
    liveloss.update(logs)
    liveloss.draw()

ValueError: too many values to unpack (expected 2)

0.5992
